In [34]:
input_parameters = 5
layer_size = 5
layer_count = 3
classification_size = 3
learning_rate = 0.001

In [35]:
from sampo.scheduler.selection.neural_net import NeuralNet, load_dataset

model = NeuralNet(input_parameters, layer_size, layer_count, classification_size)
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0792, -0.0727,  0.1207, -0.4239, -0.0794],
         [ 0.1012, -0.3573, -0.1253, -0.3287,  0.0745],
         [ 0.3024, -0.0999, -0.3640, -0.2334, -0.3335],
         [ 0.0758,  0.2433, -0.1252,  0.1163,  0.0963],
         [-0.2260, -0.0315, -0.4085,  0.4144, -0.1720]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1337, -0.3363, -0.2376,  0.1510,  0.3093], requires_grad=True)]

In [36]:
# for i in range(10):
#     print(f'Loop {i}/10')
x_train, x_test, y_train, y_test = load_dataset('dataset.csv')
model.fit(x_train, y_train, 100)
list(model.parameters())

Epoch [1/100], Step [100/112], Loss: 0.6948
Epoch [2/100], Step [100/112], Loss: 0.6924
Epoch [3/100], Step [100/112], Loss: 0.6251
Epoch [4/100], Step [100/112], Loss: 0.6026
Epoch [5/100], Step [100/112], Loss: 0.5942
Epoch [6/100], Step [100/112], Loss: 0.6335
Epoch [7/100], Step [100/112], Loss: 0.6131
Epoch [8/100], Step [100/112], Loss: 0.5823
Epoch [9/100], Step [100/112], Loss: 0.5915
Epoch [10/100], Step [100/112], Loss: 0.5821
Epoch [11/100], Step [100/112], Loss: 0.5851
Epoch [12/100], Step [100/112], Loss: 0.5859
Epoch [13/100], Step [100/112], Loss: 0.5863
Epoch [14/100], Step [100/112], Loss: 0.5836
Epoch [15/100], Step [100/112], Loss: 0.5834
Epoch [16/100], Step [100/112], Loss: 0.5874
Epoch [17/100], Step [100/112], Loss: 0.5875
Epoch [18/100], Step [100/112], Loss: 0.5916
Epoch [19/100], Step [100/112], Loss: 0.5833
Epoch [20/100], Step [100/112], Loss: 0.5823
Epoch [21/100], Step [100/112], Loss: 0.5822
Epoch [22/100], Step [100/112], Loss: 0.5883
Epoch [23/100], Ste

[Parameter containing:
 tensor([[ 0.0516, -0.1037,  0.0933, -0.4543, -0.1068],
         [ 0.1026, -0.6941, -0.0512, -0.5348,  0.1486],
         [ 0.3446, -0.1807, -0.4110, -0.2648, -0.3805],
         [ 0.0769,  0.2761, -0.1604,  0.0302,  0.0611],
         [-0.2260, -0.0315, -0.4085,  0.4144, -0.1720]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1041, -0.3961, -0.2802, -0.0536,  0.3093], requires_grad=True)]

In [37]:
y_test

[tensor([0., 1., 0.]),
 tensor([1., 0., 0.]),
 tensor([0., 0., 1.]),
 tensor([0., 1., 0.]),
 tensor([1., 0., 0.]),
 tensor([1., 0., 0.]),
 tensor([1., 0., 0.]),
 tensor([1., 0., 0.]),
 tensor([0., 1., 0.]),
 tensor([1., 0., 0.]),
 tensor([1., 0., 0.]),
 tensor([0., 0., 1.]),
 tensor([1., 0., 0.]),
 tensor([0., 0., 1.]),
 tensor([0., 0., 1.]),
 tensor([0., 1., 0.]),
 tensor([1., 0., 0.]),
 tensor([0., 1., 0.]),
 tensor([0., 1., 0.]),
 tensor([0., 1., 0.]),
 tensor([0., 1., 0.]),
 tensor([0., 0., 1.]),
 tensor([1., 0., 0.]),
 tensor([0., 1., 0.]),
 tensor([1., 0., 0.]),
 tensor([1., 0., 0.]),
 tensor([0., 0., 1.]),
 tensor([0., 0., 1.]),
 tensor([0., 0., 1.]),
 tensor([0., 0., 1.]),
 tensor([0., 1., 0.]),
 tensor([0., 1., 0.]),
 tensor([0., 0., 1.]),
 tensor([0., 1., 0.]),
 tensor([0., 0., 1.]),
 tensor([1., 0., 0.]),
 tensor([0., 0., 1.]),
 tensor([0., 0., 1.])]

In [38]:
model.test_accuracy(x_test, y_test)

Accuracy of the network on the 38 test images: 52.63157894736842 %
